In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
!wget  https://www.dropbox.com/s/n8bninud94blch4/fer2013.csv.

--2020-06-16 05:54:37--  https://www.dropbox.com/s/n8bninud94blch4/fer2013.csv.
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n8bninud94blch4/fer2013.csv. [following]
--2020-06-16 05:54:37--  https://www.dropbox.com/s/raw/n8bninud94blch4/fer2013.csv.
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc61ef0b682af5c052ca61c1f26b.dl.dropboxusercontent.com/cd/0/inline/A5tkrvoweLwI3XBkLIjXTzRFnAoVpV9L3evD6Jn9DRM1gdwDXo1ZYq9FTLL105S4vYQvcXJtNUdcOarUDdYH_pBrhkf9OXuISefy9bB7Ch-rJNuTkgOH_-tzCgMK22dKRcI/file# [following]
--2020-06-16 05:54:38--  https://uc61ef0b682af5c052ca61c1f26b.dl.dropboxusercontent.com/cd/0/inline/A5tkrvoweLwI3XBkLIjXTzRFnAoVpV9L3evD6Jn9DRM1gdwDXo1ZYq9FTLL105S4vYQvcXJtNUdcOarUDdYH_pBrhkf9OXuISefy9bB7Ch-rJNu

In [0]:
data = pd.read_csv('/content/fer2013.csv.')
data.head(5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
data.shape

(35887, 3)

In [0]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 20
width, height = 48, 48

## Question 1

In [0]:
pixels = data['pixels'].tolist()

faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    faces.append(face.astype('float32') / 255)

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

emotions = pd.get_dummies(data['emotion'])

## Question 2

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

## Question 3 & 4

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation, BatchNormalization, Dropout, MaxPool2D
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Conv2D(num_features, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:], data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(5,5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=(5,5), activation='relu', padding='same'))
model.add(Conv2D(128, kernel_size=(5,5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_labels, activation='softmax'))

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 44, 44, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        102464    
_________________________________________________________________
batch_normalization (BatchNo (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 128)       204928    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 128)       4

## Question 5

In [0]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/20
455/455 [==============================] - 33s 72ms/step - loss: 1.6966 - accuracy: 0.3401
Epoch 2/20
455/455 [==============================] - 32s 70ms/step - loss: 1.5261 - accuracy: 0.4165
Epoch 3/20
455/455 [==============================] - 32s 71ms/step - loss: 1.4138 - accuracy: 0.4593
Epoch 4/20
455/455 [==============================] - 32s 70ms/step - loss: 1.3277 - accuracy: 0.4982
Epoch 5/20
455/455 [==============================] - 32s 71ms/step - loss: 1.2616 - accuracy: 0.5214
Epoch 6/20
455/455 [==============================] - 32s 70ms/step - loss: 1.2171 - accuracy: 0.5360
Epoch 7/20
455/455 [==============================] - 32s 70ms/step - loss: 1.1755 - accuracy: 0.5528
Epoch 8/20
455/455 [==============================] - 32s 70ms/step - loss: 1.1423 - accuracy: 0.5695
Epoch 9/20
455/455 [==============================] - 32s 70ms/step - loss: 1.1076 - accuracy: 0.5807
Epoch 10/20
455/455 [==============================] - 32s 70ms/step - loss: 1.077

In [0]:
scores = model.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)

57/57 [==============================] - 1s 23ms/step - loss: 1.0528 - accuracy: 0.6258


In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
predicted = model.predict_classes(X_test)

In [0]:
y_test = np.argmax(y_test, axis=-1)

## Question 6 & 7

In [0]:
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from keras import backend as K
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.compat.v1.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.compat.v1.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph
frozen_graph = freeze_session(tf.compat.v1.keras.backend.get_session(),output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, '/content', 'fer.pb', as_text=False)